# MCP Standard Library
This notebook defines core components and utilities for the MCP framework,
including session initialization, tool management, and tool call functionality.

Chat-related elements are not included here (024_llms.ipynb)

In [ ]:
# |default_exp mcp

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

import pytest

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
##### THE PLAN #####
# sessions = list/dict (initialize servers) 
# def mcp_agent_factory(chat,[mcp_sessions]):
# tools = session.get_tools()
# Converting tools to llm format
# return Diagram (Chat(promts, tool schema), Call_tools_func(session))


# call llm (Chat(promts, tool schema)
# def call tool(args, session):


In [ ]:
# | hide
from stringdale.core import get_git_root, load_env, checkLogs, json_render

In [ ]:
# | hide
load_env()

True

In [ ]:
# | export
import asyncio
import nest_asyncio
import json
import os
from typing import Tuple, List, Dict, Any, Optional
from contextlib import asynccontextmanager,AsyncExitStack
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from mcp.client.sse import sse_client 


from anthropic import Anthropic

from openai import AsyncOpenAI

In [ ]:
nest_asyncio.apply()

In [ ]:
import asyncio
from fastmcp import Client, FastMCP
from fastmcp.client.transports import StreamableHttpTransport, PythonStdioTransport 


In [ ]:
ZAPIER_URL = os.getenv("ZAPIER_URL")
server_url = ZAPIER_URL

## Learning to work with FastMCP
https://mcp.zapier.com/mcp/servers/ad7f4d07-9f0d-4bfb-bcf5-eb210fc68d62/

In [ ]:
###### Example for sse from Zapier (with fastmcp)

transport = StreamableHttpTransport(server_url)

# Initialize the client with the transport
client = Client(transport=transport)

async with client:
    await client.ping()
    tools = await client.list_tools()
    print(tools)




[Tool(name='add_tools', title=None, description='Add new actions to your MCP provider', inputSchema={'type': 'object', 'properties': {}, 'required': []}, outputSchema=None, icons=None, annotations=ToolAnnotations(title='Add tools', readOnlyHint=None, destructiveHint=None, idempotentHint=None, openWorldHint=None), meta=None), Tool(name='edit_tools', title=None, description='Edit your existing MCP provider actions', inputSchema={'type': 'object', 'properties': {}, 'required': []}, outputSchema=None, icons=None, annotations=ToolAnnotations(title='Edit tools', readOnlyHint=None, destructiveHint=None, idempotentHint=None, openWorldHint=None), meta=None), Tool(name='google_calendar_find_events', title=None, description='Finds events in your calendar. Returns up to 25 matching events.', inputSchema={'type': 'object', 'properties': {'instructions': {'type': 'string', 'description': 'Instructions for running this tool. Any parameters that are not given a value will be guessed based on the instr

In [ ]:
# # TODO: what's the matter? why doesn't work?
# # For Python scripts
# server_script_path = get_git_root()/ "stringdale/mcp_server.py"
# transport = PythonStdioTransport(
#     script_path=server_script_path,
#     # Optional: custom environment variables
#     env={"SOME_ENV": "value"},
#     # Optional: custom Python executable
#     # python_executable="python3"
# )

# # Initialize the client with the transport
# client = Client(transport=transport)

# async with client:
#     await client.ping()
#     tools = await client.list_tools()
#     print(tools)

## Init MCP session and retrieve tools function


https://github.com/stepanogil/mcp-sse-demo/blob/master

In [ ]:
# | export
from fastmcp import Client
from fastmcp.client.transports import StreamableHttpTransport, PythonStdioTransport
from stringdale.core import get_git_root
from typing import Tuple, List

async def init_mcp_session_and_tools(config: dict) -> Tuple[Client, List]:
    """Init MCP client & fetch tools via HTTP/SSE or stdio transport (auto-detected from config).
    Explicit transport definition for more complex use cases (custom headers, authorization, etc.)

    Args:
        config: Dict with "url" or "server_script_path" and optional params.

    Returns:
        (client, tools): FastMCP Client & list of tools.
    """
    # Check if this is a multi-server configuration
    if "mcpServers" in config:
        # Multi-server mode: pass config directly to Client
        # FastMCP handles the multi-server setup internally
        client = Client(config)
        
        async with client:
            tools = await client.list_tools()
            return client, tools
    # Auto-detect transport type from config
    if "url" in config:
        # HTTP/SSE transport
        transport = StreamableHttpTransport(
            url=config["url"],
            headers=config.get("headers", {}),
        )
    elif "server_script_path" in config:
        # Stdio transport
        server_script_path = config["server_script_path"]
        # Check if the server_script_path is relative (from the main folder) and join with repo root if needed
        from pathlib import Path
        if not Path(server_script_path).is_absolute():
            server_script_path = get_git_root() / server_script_path
        
        transport = PythonStdioTransport(
            script_path=server_script_path,
            env=config.get("env", None),
        )
    else:
        raise ValueError(
            "Config must contain either 'url' (for HTTP/SSE) or "
            "'server_script_path' (for stdio transport)"
        )
    
    # Initialize the client with the transport
    client = Client(transport=transport)
    
    # Connect and get tools
    # Note: FastMCP Client manages connection lifecycle, so we need to use async context
    # But since we're returning the client, the caller will manage the context
    async with client:
        tools = await client.list_tools()
        # Return client and tools
        # Note: The client needs to stay in async context, so caller should use:
        # async with client: ... or we return it differently
        return client, tools

Testing it with openai

In [ ]:
# Example usage of init_mcp_client_and_tools

# Define your config for HTTP transport
config = {
    "url": ZAPIER_URL,  # Replace with your MCP server URL
    # "headers": {"Authorization": "Bearer ..."}, # Optional
    # "timeout": 30.0, # Optional
}
session, mcp_tools = await init_mcp_session_and_tools(config)

# Note: client is still inside context and should be used within async with if you want to make calls
for i, tool in enumerate(mcp_tools):
    print(f"Tool {i+1}: {tool}")


Tool 1: name='add_tools' title=None description='Add new actions to your MCP provider' inputSchema={'type': 'object', 'properties': {}, 'required': []} outputSchema=None icons=None annotations=ToolAnnotations(title='Add tools', readOnlyHint=None, destructiveHint=None, idempotentHint=None, openWorldHint=None) meta=None
Tool 2: name='edit_tools' title=None description='Edit your existing MCP provider actions' inputSchema={'type': 'object', 'properties': {}, 'required': []} outputSchema=None icons=None annotations=ToolAnnotations(title='Edit tools', readOnlyHint=None, destructiveHint=None, idempotentHint=None, openWorldHint=None) meta=None
Tool 3: name='google_calendar_find_events' title=None description='Finds events in your calendar. Returns up to 25 matching events.' inputSchema={'type': 'object', 'properties': {'instructions': {'type': 'string', 'description': 'Instructions for running this tool. Any parameters that are not given a value will be guessed based on the instructions.'}, '

Ok, now let's use one of the tools to make sure llm can work with them :)

First OpenAi

In [ ]:
client = AsyncOpenAI()

# allows async functions to run in jupyter notebook
nest_asyncio.apply()

# initialize the Gmail MCP client
gmail_mcp_client = session

tools = [{
"type": "function",
"function": {
    "name": tool.name,
    "description": tool.description,
    "parameters": tool.inputSchema
}
} for tool in mcp_tools]

user_input = "What events do I have on 28th of November of this year (2025)?"
# 1st LLM call to determine which tool to use
response = await client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": user_input}],
    tools=tools
)

In [ ]:
if response.choices[0].message.tool_calls:        
    tool_name = response.choices[0].message.tool_calls[0].function.name
    tool_args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
    print(f"Tool Used: {tool_name}, Arguments: {tool_args}")

    # execute the tool called by the LLM
    async with gmail_mcp_client:
        tool_response = await gmail_mcp_client.call_tool(tool_name, tool_args)
        tool_response_text = tool_response.content[0].text    


Tool Used: google_calendar_find_events, Arguments: {'instructions': 'Find all events on 28th of November 2025', 'start_time': '2025-11-28T00:00:00', 'end_time': '2025-11-28T23:59:59'}


Now the same thing with Anthropic

In [ ]:
from anthropic import AsyncAnthropic

# Initialize Anthropic client (async)
client = AsyncAnthropic()

# allows async functions to run in jupyter notebook
nest_asyncio.apply()

# initialize the MCP client (same as before)
gmail_mcp_client = session

# Convert tools to Anthropic format
# Anthropic uses "input_schema" instead of "parameters"
tools = [{
    "name": tool.name,
    "description": tool.description,
    "input_schema": tool.inputSchema  # Note: "input_schema" not "parameters"
} for tool in mcp_tools]

user_input = "What events do I have on 28th of November of this year (2025)?"

# 1st LLM call to determine which tool to use
response = await client.messages.create(
    model="claude-3-haiku-20240307",  # changed to a cheaper option
    messages=[{"role": "user", "content": user_input}],
    tools=tools,
    max_tokens=1024
)


In [ ]:
response.content

[TextBlock(citations=None, text="Okay, let's use the ", type='text'),
 ToolUseBlock(id='toolu_01G7h3YrL42zT55wQuEhJpBr', input={'instructions': 'Find events on November 28, 2025.', 'start_time': '2025-11-28T23:59:00', 'end_time': '2025-11-28T00:00:00'}, name='google_calendar_find_events', type='tool_use')]

In [ ]:

# Extract tool use from response
tool_use_block = None
for block in response.content:
    if block.type == "tool_use":
        tool_use_block = block
        break

print(tool_use_block)

tool_name = tool_use_block.name
tool_args = tool_use_block.input  # Already a dict, no need to parse JSON
print(f"Tool Used: {tool_name}, Arguments: {tool_args}")

# Execute the tool called by the LLM
async with gmail_mcp_client:
    tool_response = await gmail_mcp_client.call_tool(tool_name, tool_args)
    tool_response_text = tool_response.content[0].text


ToolUseBlock(id='toolu_01G7h3YrL42zT55wQuEhJpBr', input={'instructions': 'Find events on November 28, 2025.', 'start_time': '2025-11-28T23:59:00', 'end_time': '2025-11-28T00:00:00'}, name='google_calendar_find_events', type='tool_use')
Tool Used: google_calendar_find_events, Arguments: {'instructions': 'Find events on November 28, 2025.', 'start_time': '2025-11-28T23:59:00', 'end_time': '2025-11-28T00:00:00'}


In [ ]:
tool_response_text

'{"results":[{"kind":"calendar#event","etag":"\\"3478771113680000\\"","id":"0o8hmc4bn6uqn1h58gafeqqmog_20251128","status":"confirmed","htmlLink":"https://www.google.com/calendar/event?eid=MG84aG1jNGJuNnVxbjFoNThnYWZlcXFtb2dfMjAyNTExMjggb2xnYS5hLnNvbGRhdGVua29AbQ&ctz=America/Vancouver","created":"2024-12-06T02:23:40.000Z","updated":"2025-02-12T18:39:16.840Z","summary":"Дима Зицер\'s birthday","creator":{"email":"olga.a.soldatenko@gmail.com","self":true},"organizer":{"email":"olga.a.soldatenko@gmail.com","self":true},"start":{"date":"2025-11-28","date_pretty":"Nov 28, 2025","dateTime_pretty":"Nov 28, 2025 12:00AM","dateTime":"2025-11-28","time":""},"end":{"date":"2025-11-29","date_pretty":"Nov 29, 2025","dateTime_pretty":"Nov 29, 2025 12:00AM","dateTime":"2025-11-29","time":""},"recurringEventId":"0o8hmc4bn6uqn1h58gafeqqmog","originalStartTime":{"date":"2025-11-28"},"transparency":"transparent","visibility":"private","iCalUID":"0o8hmc4bn6uqn1h58gafeqqmog@google.com","sequence":0,"reminde

In [ ]:
tool_name

'google_calendar_find_events'

In [ ]:
tool_args

{'instructions': 'Find events on November 28, 2025.',
 'start_time': '2025-11-28T23:59:00',
 'end_time': '2025-11-28T00:00:00'}

In [ ]:
tool_response_text

'{"results":[{"kind":"calendar#event","etag":"\\"3478771113680000\\"","id":"0o8hmc4bn6uqn1h58gafeqqmog_20251128","status":"confirmed","htmlLink":"https://www.google.com/calendar/event?eid=MG84aG1jNGJuNnVxbjFoNThnYWZlcXFtb2dfMjAyNTExMjggb2xnYS5hLnNvbGRhdGVua29AbQ&ctz=America/Vancouver","created":"2024-12-06T02:23:40.000Z","updated":"2025-02-12T18:39:16.840Z","summary":"Дима Зицер\'s birthday","creator":{"email":"olga.a.soldatenko@gmail.com","self":true},"organizer":{"email":"olga.a.soldatenko@gmail.com","self":true},"start":{"date":"2025-11-28","date_pretty":"Nov 28, 2025","dateTime_pretty":"Nov 28, 2025 12:00AM","dateTime":"2025-11-28","time":""},"end":{"date":"2025-11-29","date_pretty":"Nov 29, 2025","dateTime_pretty":"Nov 29, 2025 12:00AM","dateTime":"2025-11-29","time":""},"recurringEventId":"0o8hmc4bn6uqn1h58gafeqqmog","originalStartTime":{"date":"2025-11-28"},"transparency":"transparent","visibility":"private","iCalUID":"0o8hmc4bn6uqn1h58gafeqqmog@google.com","sequence":0,"reminde

In [ ]:
#It's for openai 
client = AsyncOpenAI()
# 2nd LLM call to determine final response
res = await client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": user_input},
        {"role": "function", "name": tool_name, "content": tool_response_text},
    ]        
)

response = res.choices[0].message.content
response


"On November 28, 2025, you have the following events scheduled:\n\n1. **Dima Зицер's Birthday**\n   - **Time:** All day event (24 hours starting from November 28, 2025)\n   - **Link:** [View Event](https://www.google.com/calendar/event?eid=MG84aG1jNGJuNnVxbjFoNThnYWZlcXFtb2dfMjAyNTExMjggb2xnYS5hLnNvbGRhdGVua29AbQ&ctz=America/Vancouver)\n\n2. **Zoom Mom**\n   - **Time:** November 28, 2025, from 9:00 AM to 10:00 AM\n   - **Link:** [View Event](https://www.google.com/calendar/event?eid=MnVyc2I5aTcwcjBsMWEzbzlhYm02M2JzdHAgb2xnYS5hLnNvbGRhdGVua29AbQ&ctz=America/Los_Angeles)\n   - **Hangout Link:** [meet.google.com/zhm-qhhq-zyc](https://meet.google.com/zhm-qhhq-zyc)\n\n3. **Dean Eats at Uni**\n   - **Time:** November 28, 2025, from 1:00 PM to 2:00 PM\n   - **Link:** [View Event](https://www.google.com/calendar/event?eid=ZzV0aDFsNXNwY3IwZGZrNTE1dnJrc3M1bzRfMjAyNTExMjhUMjEwMDAwWiBvbGdhLmEuc29sZGF0ZW5rb0Bt&ctz=America/Vancouver)\n\nMake sure to check your calendar for any updates or additional 

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()